# Make csv files 

In [149]:
import os
import pandas as pd

def create_csv(image_dir, output_csv):
    data = []
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add more extensions if needed
                class_name = os.path.basename(root)  # Folder name is the class label
                file_path = os.path.abspath(os.path.join(root, file))  # Absolute file path
                data.append([file_path, class_name])
    
    # Save to CSV
    df = pd.DataFrame(data, columns=["File Name", "Class"])
    df.to_csv(output_csv, index=False)
    print(f"Saved {output_csv} with {len(data)} entries.")
    return df

# Define directories and output files
train_dir = r"C:\Users\Hassani\Downloads\applestomatoes\train"
test_dir = r"C:\Users\Hassani\Downloads\applestomatoes\test"
traindf=create_csv(train_dir, "train.csv")
testdf=create_csv(test_dir, "test.csv")


Saved train.csv with 294 entries.
Saved test.csv with 97 entries.


In [151]:
traindf.head()

,File Name,Class
0,C:\Users\Hassani\Downloads\applestomatoes\trai...,apples
1,C:\Users\Hassani\Downloads\applestomatoes\trai...,apples
2,C:\Users\Hassani\Downloads\applestomatoes\trai...,apples
3,C:\Users\Hassani\Downloads\applestomatoes\trai...,apples
4,C:\Users\Hassani\Downloads\applestomatoes\trai...,apples


## Python Data Loaders

In [374]:
import torch 
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset


In [454]:
from torchvision.transforms import Grayscale

grayscale_transform = Grayscale(num_output_channels=1)  # num_output_channels=1 for single-channel grayscale image

def target_transform(label):
    return 0 if label == "apple" else 1


class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        print("Im inside the constructor")
        self.df = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.label=self.df["Class"]
        

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
       
        img_path =self.df["File Name"][idx]
        
        image = read_image(img_path)
        image=grayscale_transform(image)
     
            
        print(img_path)
        label = self.df["Class"][idx]
        if self.transform:
            print("Before Transform image:",image.shape)
            image = self.transform(image)
            print("After Transform image:",image.shape)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])



In [456]:
train_csv_path = r"C:\Users\Hassani\Downloads\applestomatoes\train.csv"
test_csv_path = r"C:\Users\Hassani\Downloads\applestomatoes\test.csv"

In [458]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to 224x224
   
    transforms.Lambda(lambda x: x.float()),  # Convert to float32 for processing
    
])

train_dataset = CustomImageDataset(
    annotations_file=train_csv_path,
    img_dir=train_dir,
    target_transform=target_transform,transform=transform
)

Im inside the constructor


In [460]:


test_dataset = CustomImageDataset(
    annotations_file=test_csv_path,
    img_dir=test_dir,
    target_transform=target_transform
)

Im inside the constructor


In [462]:
from torch.utils.data import DataLoader


# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)


In [464]:
for data in train_loader:
    images, target = data

C:\Users\Hassani\Downloads\applestomatoes\train\apples\img_p1_10.jpeg
Before Transform image: torch.Size([1, 100, 100])
After Transform image: torch.Size([1, 128, 128])
C:\Users\Hassani\Downloads\applestomatoes\train\apples\img_p1_100.jpeg
Before Transform image: torch.Size([1, 234, 215])
After Transform image: torch.Size([1, 128, 128])
C:\Users\Hassani\Downloads\applestomatoes\train\apples\img_p1_107.jpeg
Before Transform image: torch.Size([1, 183, 275])
After Transform image: torch.Size([1, 128, 128])
C:\Users\Hassani\Downloads\applestomatoes\train\apples\img_p1_109.jpeg
Before Transform image: torch.Size([1, 259, 194])
After Transform image: torch.Size([1, 128, 128])
C:\Users\Hassani\Downloads\applestomatoes\train\apples\img_p1_11.jpeg
Before Transform image: torch.Size([1, 100, 100])
After Transform image: torch.Size([1, 128, 128])
C:\Users\Hassani\Downloads\applestomatoes\train\apples\img_p1_113.jpeg
Before Transform image: torch.Size([1, 233, 216])
After Transform image: torch.Si

In [466]:
pwd

'C:\\Users\\Hassani'